In this notebook we will export images from Polar front Sailbuoy data

In [1]:
import numpy as np
import win32com.client as win32
import glob
import os

# Select platform

### Iskant 2021

In [2]:
# Location of data files
ev_template = 'E:/PF_EV/PF_WeakStrong_Schools_FM.EV'
n_files = 20

data_folder = 'E:/SBIskant2021_PF/wbat/' 

export_folder = 'E:/PolarFront-Exports/Iskant2021/Export_WeakStrong_schools2021/' 

# Define variable name to export
var_name = 'Wideband frequency select 1'

# Find all data files with extension .raw for the date frame
raw_file_names = sorted(glob.glob(data_folder + 'Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210*.raw'))

cal_file = 'E:/SBIskant2021_PF/2021Cal_Iskant_PF.ecs'

gps_file = 'E:/SBIskant2021_PF/datalogger/sailbuoyiskant2021_gps_ctd_data.gps.csv'
#ind=900

### Iskant 2022


In [36]:
# Location of data files
ev_template = 'E:/PF_EV/PF_WeakStrong_Schools.EV'
n_files = 15

data_folder = 'E:/PolarFront2022/2022-05/sailbuoy/2022_polarfront_sb_iskant/wbat/' 

export_folder = 'E:/PolarFront-Exports/Iskant2022/Export_WeakStrong_schools/' 

# Define variable name to export
var_name = ' Fileset 1: Sv pings T1'

# Find all data files with extension .raw for the date frame
raw_file_names = sorted(glob.glob(data_folder + 'mpl1_200-Phase0-D20220*.raw'))

gps_file = 'E:/PolarFront2022/2022-05/sailbuoy/2022_polarfront_sb_iskant/datalogger/sbiskant2022_gps_ctd_data.gps.csv'


# Extract!


In [3]:
# Open new EV file

for ind in np.arange(0,len(raw_file_names), n_files):
    # Connect to the Echoview application
    EVApp = win32.Dispatch("EchoviewCom.EvApplication") 
    #EvApp.Minimize()

    #Create new file
    EvFile = EVApp.NewFile(ev_template)

    # Load data by chunks of nfiles
    Fileset = EvFile.Filesets.FindByName('Fileset 1')
    
    # Set calibration File
    Fileset.SetCalibrationFile(cal_file)
    
    # Load datafiles range
    for sub_ind in np.arange(0,n_files):
        Fileset.DataFiles.Add(raw_file_names[ind+sub_ind])
    #Fileset.DataFiles.Add(raw_file_names[ind])
    
    
    ## GPS file
    Fileset_gps = EvFile.Filesets.FindByName('Fileset 2')
    Fileset_gps.DataFiles.Add(gps_file)
    
    # Export file names
    image_file_name = os.path.basename(raw_file_names[ind])[:-4]
    export_file_name = os.path.basename(raw_file_names[ind])[:-4]
    
    
    #### Select thresholded sections and run school detection STRONG and WEAK ######
      # Pattern detection
    region_list = ["SSL Strong","SSL weak"]
    
    for j in np.arange(0,len(region_list)):
        
        # Detect fish tracks
        if region_list[j] == "SSL Strong":
            # Select variable
            tmp_var = EvFile.Variables.FindByName("SSL min -65")
            # Apply school detection settings
            EVApp.Exec("SchoolsDistanceMode =| PingNumber")
            EVApp.Exec("SchoolsMinimumTotalHeight =| 1")
            EVApp.Exec("SchoolsMinimumCandidateLengthPings =| 4")
            EVApp.Exec("SchoolsMinimumCandidateHeight =| 1")
            EVApp.Exec("SchoolsMaximumVerticalLinkDistance =| 1")
            EVApp.Exec("SchoolsMaximumHorizontalLinkDistanceInPings =| 1")
            EVApp.Exec("SchoolsMinimumTotalLengthPings =| 2")

         
            
        if region_list[j] == "SSL weak":
            # Select variable
            tmp_var = EvFile.Variables.FindByName("SSL max -75/min-80")
            # Apply school detection settings
            EVApp.Exec("SchoolsDistanceMode =| PingNumber")
            EVApp.Exec("SchoolsMinimumTotalHeight =| 10")
            EVApp.Exec("SchoolsMinimumCandidateLengthPings =| 75")
            EVApp.Exec("SchoolsMinimumCandidateHeight =| 10")
            EVApp.Exec("SchoolsMaximumVerticalLinkDistance =| 1")
            EVApp.Exec("SchoolsMaximumHorizontalLinkDistanceInPings =| 2")
            EVApp.Exec("SchoolsMinimumTotalLengthPings =| 75")

        # Remove previously detected regions
        region_temp = EvFile.RegionClasses.FindByName(region_list[j])
        region_temp.DeleteRegions(-1)
        # Detect patterns
        tmp_var.DetectSchools(region_list[j], -1, -1)

  

    #### Export and save ######
    # Export echograms
    Export_Var = EvFile.Variables.FindByName('Echogram clean for export')
    Export_Var.Properties.Display.LowerLimit=100
    Export_Var.ExportEchogramToImage(f'{export_folder}{image_file_name}.png',300,-1,-1)
    
    Class_Var = EvFile.Variables.FindByName('Echogram class for export')
    Class_Var.Properties.Display.LowerLimit=100
    Class_Var.ExportEchogramToImage(f'{export_folder}{image_file_name}class.png',500,-1,-1)
    
    # Export integration regions as csv
    Export_Var.ExportIntegrationByRegionsAll(f'{export_folder}{export_file_name}regions.csv')

    
    # Close file
    EvFile.Close()

    # Quit Echoview
    EVApp.Quit()
    
    print(raw_file_names[ind])

E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210516-T140129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210516-T190128-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210517-T000129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210517-T050129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210517-T100129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210517-T150128-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210517-T200129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210518-T010128-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210518-T060130-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210518-T110142-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES2

E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210603-T120129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210603-T170129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210603-T220129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210604-T030129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210604-T080129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210604-T130134-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210604-T180129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210604-T230128-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210605-T040129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210605-T090129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES2

E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210621-T100720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210621-T150801-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210621-T200935-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210622-T010719-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210622-T060720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210622-T110720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210622-T160720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210622-T210720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210623-T020719-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210623-T070720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES2

E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210709-T083143-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210709-T133133-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210709-T183129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210709-T233129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210710-T043129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210710-T093129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210710-T143129-0.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210710-T193720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210711-T003719-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES200-7CDK-split-PolarFront-Phase0-D20210711-T053720-1.raw
E:/SBIskant2021_PF/wbat\Mission Plan ES2

In [ ]:
np.arange(0,len(raw_file_names), n_files)

In [ ]:
np.arange(0,n_files,2)

In [15]:
ind


1358